# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started. Once you have completed the code you can download the notebook for making a submission.

In [56]:
import numpy as np
import os
from imageio import imread
from skimage.transform import resize
from skimage.util import crop
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [57]:
np.random.seed(30)
import random as rn
rn.seed(30)
from tensorflow import keras
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

**data path: /home/datasets/Project_data**

In [58]:
train_doc = np.random.permutation(open('/home/datasets/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/home/datasets/Project_data/val.csv').readlines())
#Batch size started with 200 but resource exhasted error came. Reduced to 100
batch_size = 20
img_idx = range(6,26,2)
dim_x, dim_y = 120, 120
#crop_size = 256  # Size of square crop

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [59]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    #img_idx = range(6,26,2)
    x = len(img_idx)
    y, z = dim_x, dim_y
    
    '''
    def crop_center(img, crop_width, crop_height):
        y, x = img.shape[:2]
        start_x = max(0, x//2 - crop_width//2)
        start_y = max(0, y//2 - crop_height//2)
        return img[start_y:start_y+crop_height, start_x:start_x+crop_width]
    '''
    
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size
        print(num_batches)
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                #print(folder)
                #print(batch)
                #print(batch_size)
                imgs = sorted(os.listdir(source_path+'/'+ t[folder + (batch)].split(';')[0])) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    cropped_image = crop(image, ((30, 30), (30, 30), (0, 0)))
                    image_resized = resize(cropped_image, (y,z))
                    
                    batch_data[folder,idx,:,:,0] = image_resized[:,:,0]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image_resized[:,:,1]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image_resized[:,:,2]/255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        remaining = len(folder_list) % batch_size
        if remaining > 0:
            batch_data = np.zeros((remaining, x, y, z, 3))
            batch_labels = np.zeros((remaining, 5))
           
            for folder in range(remaining):
                folder_index = num_batches + folder +1
                folder_info = t[folder_index].strip().split(';')
                folder_path = os.path.join(source_path, folder_info[0])
                
                imgs = sorted(os.listdir(folder_path))
                for idx, item in enumerate(img_idx):
                    img_path = os.path.join(folder_path, imgs[item])
                    image = imread(img_path).astype(np.float32)
                    cropped_image = crop(image, ((30, 30), (30, 30), (0, 0)))
                    image_resized = resize(cropped_image, (y, z))
                    batch_data[folder, idx] = image_resized / 255

                batch_labels[folder, int(folder_info[2])] = 1
           
            yield batch_data, batch_labels


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [60]:
curr_dt_time = datetime.datetime.now()
train_path = '/home/datasets/Project_data/train'
val_path = '/home/datasets/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 20
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 20


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [61]:
input_shape = (len(img_idx), dim_x, dim_y, 3)
input_shape

(10, 120, 120, 3)

In [62]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout, Flatten, BatchNormalization, Activation, Conv3D, MaxPooling3D, Conv2D, MaxPooling2D, TimeDistributed, GlobalAveragePooling2D, ConvLSTM2D
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras import optimizers

#write your model here


#### Convolution-2D + GRU

In [63]:
#model = Sequential()
#model.add(TimeDistributed(
#    Conv2D(32, (3,3), activation='relu'), input_shape=input_shape)
#)
#model.add(TimeDistributed(
#    MaxPooling2D((2,2)))
#)
#model.add(BatchNormalization())
#
#model.add(TimeDistributed(
#    Conv2D(64, (3,3), activation='relu'))
#)
#model.add(TimeDistributed(
#    MaxPooling2D((2,2)))
#)
#model.add(BatchNormalization())
#
#model.add(TimeDistributed(GlobalAveragePooling2D()))
#model.add(TimeDistributed(Dense(64, activation='relu')))
#model.add(BatchNormalization())
#
#model.add(GRU(128))
#model.add(BatchNormalization())
#model.add(Dense(5, activation='softmax'))

#### Convolution - 2D + GRU + dropout

In [64]:
#model = Sequential()
#model.add(TimeDistributed(
#    Conv2D(32, (3,3), activation='relu'), input_shape=input_shape)
#)
#model.add(TimeDistributed(
#    MaxPooling2D((2,2)))
#)
#model.add(BatchNormalization())
#model.add(Dropout(0.2))
#
#model.add(TimeDistributed(
#    Conv2D(64, (3,3), activation='relu'))
#)
#model.add(TimeDistributed(
#    MaxPooling2D((2,2)))
#)
#model.add(BatchNormalization())
#model.add(Dropout(0.2))
#
#model.add(TimeDistributed(GlobalAveragePooling2D()))
#model.add(TimeDistributed(Dense(64, activation='relu')))
#model.add(BatchNormalization())
#model.add(Dropout(0.2))
#
#model.add(GRU(128))
#model.add(BatchNormalization())
#model.add(Dense(5, activation='softmax'))

### Convolution-3D Simple

In [65]:
#model = Sequential()
#model.add(Conv3D(32, kernel_size=3, activation='relu', input_shape=input_shape, padding='same'))
#model.add(MaxPooling3D(pool_size=2))
#
#model.add(Conv3D(64, kernel_size=3, activation='relu', padding='same'))
#model.add(MaxPooling3D(pool_size=2))
#
#model.add(Flatten())
#model.add(Dense(256, activation='relu'))
#model.add(Dense(5, activation='softmax'))

#### Convolution-3D

In [66]:
# Getting issues while setting up Pool size, calculating and handling the same adjusting
#model = Sequential()
#
#model.add(Conv3D(32, kernel_size=3, activation='relu', input_shape=input_shape, padding='same'))
#model.add(Conv3D(64, kernel_size=3, activation='relu'))
#model.add(MaxPooling3D(pool_size=(1, 2, 2)))
#
#model.add(Conv3D(128, kernel_size=3, activation='relu', padding='same'))
#model.add(MaxPooling3D(pool_size=(1, 2, 2)))
#
#model.add(Conv3D(256, kernel_size=(3,3,3), activation='relu', padding='same'))
#model.add(MaxPooling3D(pool_size=(2, 2, 2)))
#
#model.add(Conv3D(512, kernel_size=(3, 3, 3), activation='relu', padding='same'))
#model.add(Conv3D(512, kernel_size=(3, 3, 3), activation='relu', padding='same'))
#model.add(MaxPooling3D(pool_size=(2, 2, 2)))
#
#model.add(Flatten())
#model.add(Dense(512, activation='relu'))
#model.add(Dense(5, activation='softmax'))

#### Convolution-3D with dropouts

In [67]:
#model = Sequential()
#
#model.add(Conv3D(32, kernel_size=3, activation='relu', input_shape=input_shape, padding='same'))
#model.add(Conv3D(64, kernel_size=3, activation='relu', padding='same'))
#model.add(MaxPooling3D(pool_size=(2, 2, 2)))
#model.add(BatchNormalization())
#model.add(Dropout(0.5))
#
#model.add(Conv3D(128, kernel_size=3, activation='relu', padding='same'))
#model.add(MaxPooling3D(pool_size=(1, 2, 2)))
#model.add(BatchNormalization())
#model.add(Dropout(0.5))
#
#model.add(Conv3D(256, kernel_size=(1, 3, 3), activation='relu', padding='same'))
#model.add(MaxPooling3D(pool_size=(1, 2, 2)))
#model.add(BatchNormalization())
#model.add(Dropout(0.5))
#
#model.add(Conv3D(512, kernel_size=(1, 3, 3), activation='relu', padding='same'))
#model.add(Conv3D(512, kernel_size=(1, 3, 3), activation='relu', padding='same'))
#model.add(MaxPooling3D(pool_size=(1, 2, 2)))
#model.add(BatchNormalization())
#model.add(Dropout(0.5))
#
#model.add(Flatten())
#model.add(Dense(512, activation='relu', padding='same'))
#model.add(BatchNormalization())
#model.add(Dense(5, activation='softmax'))

#### Conv3D-reduced dropouts

In [68]:
#model = Sequential()
#
#model.add(Conv3D(32, kernel_size=3, activation='relu', input_shape=input_shape))
#model.add(Conv3D(64, kernel_size=3, activation='relu'))
#model.add(MaxPooling3D(pool_size=(2, 2, 2)))
#model.add(BatchNormalization())
#model.add(Dropout(0.2))
#
#model.add(Conv3D(128, kernel_size=3, activation='relu'))
#model.add(MaxPooling3D(pool_size=(1, 2, 2)))
#model.add(BatchNormalization())
#model.add(Dropout(0.2))
#
#model.add(Conv3D(256, kernel_size=(1, 3, 3), activation='relu'))
#model.add(MaxPooling3D(pool_size=(1, 2, 2)))
#model.add(BatchNormalization())
#model.add(Dropout(0.2))
#
#model.add(Flatten())
#model.add(Dense(512, activation='relu'))
#model.add(BatchNormalization())
#model.add(Dense(5, activation='softmax'))

#### LSTM-CONV2D

In [69]:
model = Sequential()
model.add(TimeDistributed(
    Conv2D(8, (3,3), activation='relu'), input_shape=input_shape)
)
model.add(BatchNormalization())
model.add(TimeDistributed(
    Conv2D(16, (3,3), activation='relu'))
)
model.add(BatchNormalization())
model.add(
    ConvLSTM2D(8, kernel_size = 3, return_sequences=False)
)
model.add(BatchNormalization())
model.add(TimeDistributed(
    Dense(64, activation='relu'))
)
model.add(BatchNormalization())
model.add(GlobalAveragePooling2D())
model.add(Dense(64, activation='relu'))
model.add(Dense(5, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [70]:
optimiser = optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d_8 (Conv3D)           (None, 8, 118, 118, 32)   2624      
                                                                 
 conv3d_9 (Conv3D)           (None, 6, 116, 116, 64)   55360     
                                                                 
 max_pooling3d_7 (MaxPooling  (None, 3, 58, 58, 64)    0         
 3D)                                                             
                                                                 
 batch_normalization_4 (Batc  (None, 3, 58, 58, 64)    256       
 hNormalization)                                                 
                                                                 
 dropout_3 (Dropout)         (None, 3, 58, 58, 64)     0         
                                                                 
 conv3d_10 (Conv3D)          (None, 1, 56, 56, 128)   

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [71]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [72]:
model_name = 'LSTM-Conv2D' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'LSTM-Conv2D-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto',save_freq = 'epoch')

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, min_lr=0.001, patience=5, verbose=1)
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit` method to decide the number of next() calls it need to make.

In [73]:
print(batch_size)
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
    
print(steps_per_epoch)

20
34


Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [74]:
model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  /home/datasets/Project_data/train ; batch size = 20
33
Epoch 1/20
34/34 [==============================] - ETA: 0s - loss: 0.5331 - categorical_accuracy: 0.9020Source path =  /home/datasets/Project_data/val ; batch size = 20
5
5

Epoch 00001: saving model to Conv3D-Dropout-0.2_2024-08-0617_26_29.564942/Conv3D-Dropout-0.2-00001-0.53311-0.90196-1147.50574-0.28000.h5
34/34 [==============================] - 49s 1s/step - loss: 0.5331 - categorical_accuracy: 0.9020 - val_loss: 1147.5057 - val_categorical_accuracy: 0.2800 - lr: 0.0100
Epoch 2/20
34/34 [==============================] - ETA: 0s - loss: 0.8652 - categorical_accuracy: 0.88845

Epoch 00002: saving model to Conv3D-Dropout-0.2_2024-08-0617_26_29.564942/Conv3D-Dropout-0.2-00002-0.86515-0.88839-123.11698-0.18000.h5
34/34 [==============================] - 46s 1s/step - loss: 0.8652 - categorical_accuracy: 0.8884 - val_loss: 123.1170 - val_categorical_accuracy: 0.1800 - lr: 0.0100
Epoch 3/20
33/34 [===================

# 